### Imports and Installing

In [135]:
!pip install tabula-py==0.3.0
!pip install PyPDF2

  Running setup.py bdist_wheel for tika: started
  Running setup.py bdist_wheel for tika: finished with status 'done'
  Stored in directory: C:\Users\Jason\AppData\Local\pip\Cache\wheels\b4\db\8a\3a3f0c0725448eaa92703e3dda71e29dc13a119ff6c1036848
Successfully built tika


You are using pip version 18.1, however version 19.0.2 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


You are using pip version 18.1, however version 19.0.2 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [1]:
%run ../imports.py

C:\Users\Jason\AppData\Local\conda\conda\envs\DSI-6\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
from bs4 import BeautifulSoup as bs
import requests, tabula, io, os, time
from tabula import read_pdf, convert_into
import datetime
from PyPDF2 import PdfFileReader, PdfFileWriter

import warnings
warnings.filterwarnings("ignore")

# from urllib.request import urlopen
# from tika import parser  # tika had even more errors than tabula
# !pip install tika

# used tabula which is a Java wrapper since PyPDF2 and processing directly from pdf url didn't work
# With tabula, the url pdf is written to a temp pdf file then processed into a dataframe in Java

### Extracting Last 2 Minutes Reports for March 2015 to June 2017

In [3]:
main_url1 = 'https://official.nba.com/nba-last-two-minute-reports-archive/'
main_url2 = 'https://official.nba.com/2017-18-nba-officiating-last-two-minute-reports/'
raw_headers = ['Period','Time Call Type','Committing Player','Disadvantaged Player','Unnamed: 4',
               'Review Decision','Video','Date','Score']
df = pd.DataFrame(columns=raw_headers)
df_errors = pd.DataFrame(columns=raw_headers)

def extract_df(main_url):
    res = requests.get(main_url)
    soup = bs(res.content, 'lxml')

    div = soup.find('div', {'class': 'entry-content'})

    resume = True
    c = 0
    for row in div.find_all('p'):
        c += 1
        game = {}
        links = row.find_all('a')

        if links == []:
            pass
        else:
        # only process page elements that have links in them (assumed href are only in <a> elements)
            for link in links:
#                 if '4/2016/11/76ers-109-Pacers-105-OT.pdf' in link.attrs['href'] :
#                     print(link)
#                     resume = True
#                 elif '4/2015/06/L2M-GSW-CLE-6-9-15-FINALS-GAME-3.pdf' in link.attrs['href'] : 
#                     print(link)
#                     resume = False
                if resume:
                    score = link.text
                    score = score.replace(' ', '')
                    score = score.replace(',', '_')
                    link = link.attrs['href']
                    if 'https://ak-static.cms.nba.com/' in link:  
                        # narrows results to only pdf game score links
                        try :
                            date = link.split('/')[-1]
                            date = date.split('-')[3:6]
                            # split 1 to 3 are 'L2M', home team, visiting team, after that are date values:
                            date_str = date[0] + '-' + date[1] + '-' + date[2].strip('.pdf')
                            date[2] = date[2].strip('.pdf')
                            file_name = score + '_' + date_str
                        except :
                            file_name = link
                        print(file_name)

                        try : raw_content = requests.get(link).content
                        except :
                            try :
                                time.sleep(5)
                                raw_content = requests.get(link).content
                            except :
                                time.sleep(45)
                                raw_content = requests.get(link).content
                        try :
    #                         parsed = parser.from_file(link)
    #                         print(parsed["content"]) # a lot of ordering errors with tika parser
                            with io.BytesIO(raw_content) as open_pdf_file:
                                pdfFile = PdfFileReader(open_pdf_file, strict=False)
                                temp_pdf = PdfFileWriter()
                                temp_pdf.appendPagesFromReader(pdfFile)
                                temp_pdf.write(open('temp_pdf.pdf', 'wb'))
                            df_game = tabula.read_pdf('temp_pdf.pdf') # some pdf errors with tabula
                            df_game['Date'] = datetime.date(int(date[2]), int(date[0]), int(date[1]))
                            df_game['Score'] = score
                            global df
                            global df_errors
    #                         print(df_game.columns, len(df_game.columns))
    #                         print((df.columns==df_game.columns).all())
                            if (len(df_game.columns) == len(df.columns)) & (df_game.columns == df.columns).all():
                                df = df.append(df_game, ignore_index=True)
                            else:
                                df_game.loc[-1,:] = df_game.columns
    #                             df_game.columns[0:9] = df.columns[0:9]
                                df_errors = df_errors.append(df_game, ignore_index=True)
                                os.rename('temp_pdf.pdf', file_name + '.pdf')

                        except Exception as err : 
                            try :
                                print(err)
                                print(df.shape)
                                print(df_game.shape)
                            except : pass

In [24]:
extract_df(main_url2)

Warriors110_Cavaliers102_06-06-2018
Lengths must match to compare
(0, 9)
(33, 11)
Warriors124_Cavaliers114(OT)_05-31-2018
Lengths must match to compare
(0, 9)
(34, 10)
Rockets98_Warriors94_05-24-2018
Lengths must match to compare
(0, 9)
(34, 10)
Rockets95_Warriors92_05-22-2018
Lengths must match to compare
(0, 9)
(33, 10)
Celtics114_76ers112_05-09-2018


KeyboardInterrupt: 

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5188 entries, 0 to 5187
Data columns (total 9 columns):
Period                  5153 non-null object
Time Call Type          5188 non-null object
Committing Player       2501 non-null object
Disadvantaged Player    2137 non-null object
Unnamed: 4              128 non-null object
Review Decision         2558 non-null object
Video                   2670 non-null object
Date                    5188 non-null object
Score                   5188 non-null object
dtypes: object(9)
memory usage: 364.9+ KB


In [18]:
# df.to_csv("L2M_archive.csv", index=False, compression='gzip')
# df.to_csv("L2M_through_Raptors118_Knicks107_11-12-16.csv", index=False, compression='gzip')

### Trying to Parse 17-18 Season PDF's which has New Structure & Graphics

In [25]:
# pdf's from 17-18 season onward have new graphics, must parse differently
df_game = tabula.read_pdf('temp_pdf.pdf')
df_game.head()
# Formatting and structuring after parsing is more frequently off
# Decision was to use just 15-16 and 16-17 sets and not 17-18 dataset

,Unnamed: 0,02:01.1,Unnamed: 2,0 1:50.0,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8
0,NaN,02:01.1 To,01:50.0,NaN,NaN,NaN,NaN,NaN,NaN
1,Q4,01:52.0,Foul: Personal,NaN,Ben Simmons,Jayson Tatum,NaN,CNC,Video
2,ment:,Simmons (PHI) reaches in and makes marginal co...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Q4,01:50.0,Foul: Shooting,NaN,Ben Simmons,Jayson Tatum,NaN,CC,Video
4,ment:,Simmons (PHI) makes contact to Tatum's (BOS) a...,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Last Automated Option to Parse PDF's with Errors

In [4]:
files = os.listdir('./data/')
df = pd.DataFrame(columns=raw_headers)
df_errors = pd.DataFrame(columns=raw_headers)

for file in files:
    if '.pdf' not in file:
        continue
    else:
        print(file)
        try:
            # most errors are from java parser using first data row as header, fixing those pdfs
            df_game = tabula.read_pdf('./data/' + file)
            df_game.loc[-1,:] = df_game.columns
            file = file.strip('.pdf')
            file = file.split('_')
            date = file[2].split('-')
            df_game['Date'] = datetime.date(int(date[2]), int(date[0]), int(date[1]))
            df_game['Score'] = file[0] + '_' + file[1]
            
#             print(df_game.columns)
            df_game.columns = df.columns
            if (len(df_game.columns) == len(df.columns)) & (df_game.columns == df.columns).all():
                df = df.append(df_game, ignore_index=True)
        except Exception as err : 
            print(err)


76ers104_Warriors106_03-14-17.pdf
76ers105_TrailBlazers108_3-26-16.pdf
76ers109_Pacers105_11-11-16.pdf
76ers124_Nuggets122_12-30-16.pdf
Blazers106_Timberwolves101_11-2-15.pdf
Bucks101_Suns95_12-20-15.pdf
Bucks105_Knicks104_01-04-17.pdf
Bucks106_Bulls101_1-12-16.pdf
Bucks108_Cavaliers105_11-14-15.pdf
Bucks108_Hawks101_1-15-16.pdf
Bucks113_Magic110_4-1-16.pdf
Bucks91_76ers87_11-4-15.pdf
Bucks91_Pistons92_3-21-16.pdf
Bulls101_Nets99_12-28-16.pdf
Bulls102_Kings99 _01-21-17.pdf
Bulls102_Pacers100_12-30-15.pdf
Bulls103_Rockets100_3-31-16.pdf
Bulls107_Kings102_2-3-16.pdf
Bulls115_76ers111_1-14-16.pdf
Bulls115_Raptors113_1-3-16.pdf
Bulls121_Pelicans116_4-11-16.pdf
Bulls126_Lakers115_2-21-16.pdf
Bulls92_Magic87_11-1-15.pdf
Bulls92_Spurs89_11-30-151.pdf
Bulls93_TrailBlazers88_11-24-15.pdf
Bulls98_Pelicans94_12-12-15.pdf
Bulls_Mavericks_12-26-15.pdf
Cavaliers100_Pacers96_2-29-16.pdf
Cavaliers102_Sixers101_11-5-16.pdf
Cavaliers106_Pistons101_4-17-16.pdf
Cavaliers109_Pacers108_04-15-17.pdf
Cavalier

Raptors96_Heat92_5-5-16.pdf
Raptors96_Heat94_04-07-17.pdf
Raptors97_Spurs94_12-9-15.pdf
Raptors98_Grizzlies85_2-21-16.pdf
Raptors99_Grizzlies95_4-1-16.pdf
Raptors99_Heat91_5-11-16.pdf
'utf-8' codec can't decode byte 0x92 in position 19: invalid start byte
Raptors_Pacers_3-17-16.pdf
Raptors_Timberwolves_2-10-16.pdf
Rockets100_Pelicans95_3-2-16.pdf
Rockets102_Hornets95_12-21-15.pdf
Rockets107_Pacers103_1-10-16.pdf
Rockets107_Timberwolves104_1-13-16.pdf
Rockets109_Wizards103_12-9-15.pdf
Rockets110_Thunder105_11-2-15.pdf
Rockets112_Pelicans111_1-25-16.pdf
Rockets113_Raptors107_3-6-16.pdf
Rockets113_Thunder109_04-23-17.pdf
Rockets116_76ers114 _11-27-15.pdf
Rockets116_Knicks111_11-29-15.pdf
Rockets117_Pelicans107_03-24-17.pdf
Rockets118_Thunder116_01-05-17.pdf
Rockets119_TrailBlazers105_2-25-16.pdf
Rockets120_Kings113_12-5-15.pdf
Rockets88_Spurs84_12-25-15.pdf
Rockets93_Jazz91_1-4-16.pdf
Rockets99_Pistons96_11-21-16.pdf
Rockets_Pelicans_12-26-15.pdf
Sixers108_Nets107_12-18-16.pdf
Spurs100_Kn

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10037 entries, 0 to 10036
Data columns (total 9 columns):
Period                  9782 non-null object
Time Call Type          9953 non-null object
Committing Player       4904 non-null object
Disadvantaged Player    3091 non-null object
Unnamed: 4              2060 non-null object
Review Decision         4660 non-null object
Video                   5238 non-null object
Date                    10037 non-null object
Score                   10037 non-null object
dtypes: object(9)
memory usage: 705.8+ KB


In [34]:
df['Period'].value_counts()

Q4                                                                                                                                                                  1215
Comment:                                                                                                                                                             662
mment:                                                                                                                                                               652
Q5                                                                                                                                                                   198
Period                                                                                                                                                               103
Q6                                                                                                                                                         

In [33]:
df['Unnamed: 4'].value_counts()

CNC                        73
Review Decision            50
CC                         40
Disadvantaged Player       39
Unnamed: 4                 16
INC                        14
Marc Gasol                 11
Paul Millsap               11
Jimmy Butler               11
Pau Gasol                  10
Mike Conley                 9
Tony Allen                  9
Isaiah Thomas               8
Chris Paul                  8
Austin Rivers               8
Robert Covington            7
Jeff Teague                 7
Goran Dragic                7
DeMarcus Cousins            7
LeBron James                6
Kent Bazemore               6
Jamal Crawford              6
Hassan Whiteside            6
Al Horford                  6
George Hill                 6
Blake Griffin               6
Dwyane Wade                 5
Evan Fournier               5
Joel Embiid                 5
Josh Richardson             5
Andrew Harrison             5
Zach Randolph               5
Tyler Johnson               5
Derrick Ro

In [36]:
df['Review Decision'].value_counts()

CNC                579
CC                 510
INC                120
Review Decision     54
Unnamed: 5          50
IC                  19
Name: Review Decision, dtype: int64

In [35]:
df['Video'].value_counts()

Video    1536
Name: Video, dtype: int64

In [7]:
df.to_csv("L2M_errors_fixed.csv", index=False, compression='gzip')

### Formatting Dataframe which Collected Tables with Obvious Errors

In [25]:
df = pd.read_csv("L2M_errors_fixed.csv", index_col=None, compression='gzip')
df.head()

,Period,Time Call Type,Committing Player,Disadvantaged Player,Unnamed: 4,Review Decision,Video,Date,Score
0,Q4,01:38.0 Foul: Personal,Timothe Luwawu-Cabarrot,NaN,Klay Thompson,CNC,Video,0017-03-14,76ers104_Warriors106
1,mment:,Luwawu-Cabarrot (PHI) makes marginal contact w...,NaN,NaN,NaN,NaN,NaN,0017-03-14,76ers104_Warriors106
2,Q4,01:21.0 Foul: Personal,Matt Barnes,NaN,Robert Covington,CC,Video,0017-03-14,76ers104_Warriors106
3,mment:,Barnes (GSW) makes contact to Covington's (PHI...,NaN,NaN,NaN,NaN,NaN,0017-03-14,76ers104_Warriors106
4,Q4,01:15.0 Foul: Offensive,Dario Saric,NaN,Stephen Curry,CNC,Video,0017-03-14,76ers104_Warriors106


In [110]:
df = pd.read_csv("L2M_errors_fixed.csv", index_col=None, compression='gzip')

review_types = ['CNC', 'CC', 'INC', 'IC']
foul_types = ['Personal', '24 Second', 'Shooting', 'Loose Ball', 'Offensive', 'Traveling']
# The major types of analyzable fouls, will only be investigating this subset

def align_to_columns(df):
    debug = ''
    
    try:
        df_errors = pd.DataFrame()
        
        if (df.columns != raw_headers).all():
            print('column header error')
            df.columns = raw_headers

        # For loop to add all comment text to its matching row, the one above
        for idx, row in df.iterrows():
            if type(row['Period']) == float:
                continue
            elif 'Period' in row['Period']:
                continue
            elif (row['Period'] in 'Comment:'):
                df_errors.loc[idx-1, 'Comment'] = row['Time Call Type']
                continue
            elif (row['Period'].split()[0] in 'Comment:'):
                df_errors.loc[idx-1, 'Comment'] = str(row['Period']) + ' ' + str(row['Time Call Type'])
                continue
                
        for idx, row in df.iterrows():
            # Processing column 'Period'
            debug = row['Period']
            if type(row['Period']) == float:
                continue
            elif 'Period' in row['Period']:
                continue
            elif row['Period'] in ['Q4','Q5','Q6']: # possible correct values for this column
                df_errors.loc[idx, 'Period'] = row['Period']
            elif ('Q4' in row['Period']):
                df_errors.loc[idx, 'Period'] = 'Q4'
            elif ('Q5' in row['Period']):
                df_errors.loc[idx, 'Period'] = 'Q5'
            elif ('Q6' in row['Period']):
                df_errors.loc[idx, 'Period'] = 'Q6'
            else:
                continue

            if (df_errors.loc[idx, 'Period'] in ['Q4','Q5','Q6']):
                df_errors.loc[idx, 'Soup'] = ''
                for cell in row.values:
                    df_errors.loc[idx, 'Soup'] += str(cell) + ' '
                    
            # Processing for column 'Time' and 'Foul' (foul type)
                match = re.search('\d\d:\d\d.\d', df_errors.loc[idx, 'Soup'])
                if match:
                    time = match.group()
                    df_errors.loc[idx, 'Time'] = time
                for foul in foul_types:
                    if foul in df_errors.loc[idx, 'Soup']:
                        df_errors.loc[idx, 'Foul'] = foul
                                
            # Processing for columns 'Fouler' and 'Foulee'
            name1 = str(df.loc[idx, 'Committing Player'])
            name3 = str(df.loc[idx, 'Unnamed: 4']) # This column should be empty, except by parser error
            name2 = str(df.loc[idx, 'Disadvantaged Player'])
            foulee_done = False
            # If name is in correct 2nd column it is always the disadvantaged player
            if len(name2.split()) == 2: # If entry is two words will assume it is a name
                df_errors.loc[idx, 'Foulee'] = name2
                foulee_done = True
            if len(name3.split()) == 2:
                if foulee_done: # then name in this column is for fouler
                    df_errors.loc[idx, 'Fouler'] = name3
                else:
                    df_errors.loc[idx, 'Foulee'] = name3
                    foulee_done = True
            if len(name1.split()) == 2:
                if foulee_done: # then name in this column is for fouler
                    df_errors.loc[idx, 'Fouler'] = name1
                else:
                    df_errors.loc[idx, 'Foulee'] = name1
                    foulee_done = True
            if not foulee_done: pass
#                 print('no name found: ', row)
            
            review_done = False
            for review_decision in review_types:
                if review_decision in str(df_errors.loc[idx, 'Soup']):
                    df_errors.loc[idx, 'Review Decision'] = review_decision
                    review_done = True
            if not review_done:  # if parser moved review decision to row below in table
                soup_temp = ''
                for cell in df.loc[idx+1,:].values:
                    soup_temp += str(cell) + ' '
                for review_decision in review_types:
                    if review_decision in soup_temp:
                        df_errors.loc[idx, 'Review Decision'] = review_decision  
                        review_done = True
            if not review_done:
                if 'Observable in enhanced video' in str(df_errors.loc[idx, 'Comment']):
                    df_errors.loc[idx, 'Review Decision'] = 'PINC' # for Partial Incorrect No Call 
            
            df_errors.loc[idx, 'Date'] = row['Date']
            df_errors.loc[idx, 'Score'] = row['Score']

    except ValueError as err : print(debug, err)
    return df_errors

In [111]:
df_errors = align_to_columns(df)

In [112]:
df_errors.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4909 entries, 0 to 10035
Data columns (total 10 columns):
Comment            4520 non-null object
Period             4891 non-null object
Soup               4891 non-null object
Time               4888 non-null object
Foul               4318 non-null object
Foulee             4554 non-null object
Fouler             3964 non-null object
Review Decision    4801 non-null object
Date               4891 non-null object
Score              4891 non-null object
dtypes: object(10)
memory usage: 581.9+ KB


In [113]:
df_errors[df_errors.drop(['Time','Fouler','Comment'], axis=1).isnull().any(axis=1)]

,Comment,Period,Soup,Time,Foul,Foulee,Fouler,Review Decision,Date,Score
12,Saric (PHI) is in the paint for longer than th...,Q4,Q4 00:40.0 Turnover: 3 Second Violation Dario ...,00:40.0,NaN,Dario Saric,NaN,INC,0017-03-14,76ers104_Warriors106
16,LHH shows Holmes (PHI) step out of bounds.,Q4,Q4 00:36.0 Turnover: Stepped out of Bounds Ric...,00:36.0,NaN,Richaun Holmes,NaN,CC,0017-03-14,76ers104_Warriors106
30,Saric (PHI) steps over the plane of the free t...,Q4,Q4 00:02.6 Violation: Lane Dario Saric nan nan...,00:02.6,NaN,Dario Saric,NaN,INC,0017-03-14,76ers104_Warriors106
34,Davis (POR) attempts to get in path of Smith (...,Q4,Q4 01:37.0 Turnover: Illegal Screen Ed Davis I...,01:37.0,NaN,Ish Smith,Ed Davis,CNC,0016-03-26,76ers105_TrailBlazers108
54,Grant (PHI) commits a lane violation. The free...,Q4,Q4 00:07.2 Violation: Lane Jerami Grant nan na...,00:07.2,NaN,Jerami Grant,NaN,CC,0016-03-26,76ers105_TrailBlazers108
59,Embiid (PHI) is in the paint for less than thr...,Q4,Q4 01:55.0 Turnover: 3 Second Violation Joel E...,01:55.0,NaN,Joel Embiid,NaN,CNC,0016-11-11,76ers109_Pacers105
85,Correctly called 24 second violation.,Q5,Q5 01:30.0 Turnover: 24 Second Violation nan n...,01:30.0,24 Second,NaN,NaN,CC,0016-11-11,76ers109_Pacers105
91,"After communicating with the Replay Center, th...",Q5,Q5 00:36.5 Instant Replay: Support Ruling nan ...,00:36.5,NaN,NaN,NaN,CC,0016-11-11,76ers109_Pacers105
111,LHH shows Saric (PHI) reestablishes his foot i...,Q4,Q4 00:21.7 Turnover: Stepped out of Bounds Dar...,00:21.7,NaN,Dario Saric,NaN,CNC,0016-12-30,76ers124_Nuggets122
117,The basketball is last touched by Philadelphia...,Q4,Q4 00:13.8 Stoppage: Out-of-Bounds nan nan nan...,00:13.8,NaN,NaN,NaN,CC,0016-12-30,76ers124_Nuggets122


In [114]:
df_errors['Period'].value_counts()

Q4    4332
Q5     554
Q6       5
Name: Period, dtype: int64

In [82]:
df_errors['Time'].sort_values()

7642     00:00.0
8260     00:00.1
419      00:00.1
5592     00:00.1
7730     00:00.1
7915     00:00.2
4748     00:00.3
1307     00:00.3
3500     00:00.3
2062     00:00.4
3310     00:00.4
4664     00:00.4
3608     00:00.4
4614     00:00.5
2026     00:00.5
2884     00:00.5
4540     00:00.5
8786     00:00.5
1201     00:00.6
1542     00:00.6
1237     00:00.6
8217     00:00.6
5400     00:00.7
6864     00:00.7
7020     00:00.7
5669     00:00.7
7435     00:00.7
1874     00:00.8
5936     00:00.8
1893     00:00.8
3084     00:00.8
3977     00:00.9
7964     00:00.9
4974     00:00.9
4972     00:00.9
3449     00:00.9
7804     00:00.9
658      00:00.9
8507     00:01.0
2615     00:01.0
8000     00:01.0
1199     00:01.0
6413     00:01.1
7624     00:01.1
8082     00:01.1
6106     00:01.2
529      00:01.2
8954     00:01.2
8882     00:01.2
4195     00:01.2
8561     00:01.3
5883     00:01.3
7962     00:01.3
5222     00:01.3
8563     00:01.3
7368     00:01.3
9375     00:01.4
2766     00:01.4
6255     00:01

In [115]:
df_errors['Foul'].value_counts()

Personal      1540
Shooting      1265
Offensive      852
Loose Ball     400
Traveling      227
24 Second       34
Name: Foul, dtype: int64

In [116]:
df_errors['Review Decision'].value_counts()

CNC     2466
CC      1683
INC      455
PINC     128
IC        69
Name: Review Decision, dtype: int64

In [117]:
df = df_errors[~df_errors[['Review Decision','Foulee','Comment']].isnull().any(axis=1)]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4149 entries, 0 to 10033
Data columns (total 10 columns):
Comment            4149 non-null object
Period             4149 non-null object
Soup               4149 non-null object
Time               4148 non-null object
Foul               3925 non-null object
Foulee             4149 non-null object
Fouler             3633 non-null object
Review Decision    4149 non-null object
Date               4149 non-null object
Score              4149 non-null object
dtypes: object(10)
memory usage: 356.6+ KB


In [118]:
df[df['Foul'].isnull()] # All the types of fouls of this project seem to have been extracted correctly

,Comment,Period,Soup,Time,Foul,Foulee,Fouler,Review Decision,Date,Score
12,Saric (PHI) is in the paint for longer than th...,Q4,Q4 00:40.0 Turnover: 3 Second Violation Dario ...,00:40.0,NaN,Dario Saric,NaN,INC,0017-03-14,76ers104_Warriors106
16,LHH shows Holmes (PHI) step out of bounds.,Q4,Q4 00:36.0 Turnover: Stepped out of Bounds Ric...,00:36.0,NaN,Richaun Holmes,NaN,CC,0017-03-14,76ers104_Warriors106
30,Saric (PHI) steps over the plane of the free t...,Q4,Q4 00:02.6 Violation: Lane Dario Saric nan nan...,00:02.6,NaN,Dario Saric,NaN,INC,0017-03-14,76ers104_Warriors106
34,Davis (POR) attempts to get in path of Smith (...,Q4,Q4 01:37.0 Turnover: Illegal Screen Ed Davis I...,01:37.0,NaN,Ish Smith,Ed Davis,CNC,0016-03-26,76ers105_TrailBlazers108
54,Grant (PHI) commits a lane violation. The free...,Q4,Q4 00:07.2 Violation: Lane Jerami Grant nan na...,00:07.2,NaN,Jerami Grant,NaN,CC,0016-03-26,76ers105_TrailBlazers108
59,Embiid (PHI) is in the paint for less than thr...,Q4,Q4 01:55.0 Turnover: 3 Second Violation Joel E...,01:55.0,NaN,Joel Embiid,NaN,CNC,0016-11-11,76ers109_Pacers105
111,LHH shows Saric (PHI) reestablishes his foot i...,Q4,Q4 00:21.7 Turnover: Stepped out of Bounds Dar...,00:21.7,NaN,Dario Saric,NaN,CNC,0016-12-30,76ers124_Nuggets122
123,"Prior to the inbound pass, Covington (PHI) and...",Q4,Q4 00:10.1 Foul: Away from Play Robert Covingt...,00:10.1,NaN,Danilo Gallinari,Robert Covington,CNC,0016-12-30,76ers124_Nuggets122
148,[Observable in enhanced video] After communica...,Q4,Q4 00:54.4 Instant Replay: Support Ruling Andr...,00:54.4,NaN,Andrew Wiggins,NaN,PINC,0015-11-02,Blazers106_Timberwolves101
316,Korver (ATL) intentionally kicks the ball on t...,Q5,Q5 02:13.0 Violation: Kicked Ball Kyle Korver ...,02:13.0,NaN,Kyle Korver,NaN,CC,0016-01-15,Bucks108_Hawks101


In [119]:
df = df[~df['Foul'].isnull()]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3925 entries, 0 to 10033
Data columns (total 10 columns):
Comment            3925 non-null object
Period             3925 non-null object
Soup               3925 non-null object
Time               3924 non-null object
Foul               3925 non-null object
Foulee             3925 non-null object
Fouler             3586 non-null object
Review Decision    3925 non-null object
Date               3925 non-null object
Score              3925 non-null object
dtypes: object(10)
memory usage: 337.3+ KB


### Formatting Mostly Correct Raw Dataframes

In [120]:
df2 = pd.read_csv("./data/L2M_through_Raptors118_Knicks107_11-12-16.csv", compression='gzip')
df3 = pd.read_csv("./data/L2M_archive.csv", compression='gzip')

In [123]:
df2 = df2.append(df3, ignore_index=True)
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14100 entries, 0 to 14099
Data columns (total 9 columns):
Period                  14030 non-null object
Time Call Type          14093 non-null object
Committing Player       6820 non-null object
Disadvantaged Player    5702 non-null object
Unnamed: 4              250 non-null object
Review Decision         6708 non-null object
Video                   7248 non-null object
Date                    14100 non-null object
Score                   14100 non-null object
dtypes: object(9)
memory usage: 991.5+ KB


In [127]:
df2[~df2['Unnamed: 4'].isnull()]

,Period,Time Call Type,Committing Player,Disadvantaged Player,Unnamed: 4,Review Decision,Video,Date,Score
1987,Q4,01:37.0 Foul: Offensive,Julius Randle,Bradley Beal,Bradley Beal,CNC,Video,0017-03-28,Wizards119_Lakers108
1989,Q4,01:33.0 Foul: Personal,Jordan Clarkson,Kelly Oubre Jr.,Kelly Oubre Jr.,CNC,Video,0017-03-28,Wizards119_Lakers108
1991,Q4,01:25.0 Foul: Personal,Julius Randle,Kelly Oubre Jr.,Kelly Oubre Jr.,CC,Video,0017-03-28,Wizards119_Lakers108
1993,Q4,01:23.0 Foul: Offensive,Julius Randle,Kelly Oubre Jr.,Kelly Oubre Jr.,CNC,Video,0017-03-28,Wizards119_Lakers108
2003,Q4,00:32.8 Foul: Shooting,Tyler Ennis,Otto Porter Jr.,Otto Porter Jr.,CNC,Video,0017-03-28,Wizards119_Lakers108
2783,Q4,01:50.0 Foul: Personal,Klay Thompson,Ricky Rubio,Ricky Rubio,CNC,Video,0017-03-10,Timberwolves103_Warriors102
2787,Q4,01:45.0 Foul: Shooting,Andre Iguodala,Andrew Wiggins,Andrew Wiggins,CNC,Video,0017-03-10,Timberwolves103_Warriors102
2789,Q4,01:44.0 Foul: Loose Ball,Stephen Curry,Nemanja Bjelica,Nemanja Bjelica,CNC,Video,0017-03-10,Timberwolves103_Warriors102
2791,Q4,01:41.0 Foul: Shooting,Klay Thompson,Brandon Rush,Brandon Rush,CNC,Video,0017-03-10,Timberwolves103_Warriors102
2793,Q4,01:29.0 Foul: Offensive,Stephen Curry,Nemanja Bjelica,Nemanja Bjelica,CNC,Video,0017-03-10,Timberwolves103_Warriors102


In [126]:
def get_name(row):
    if type(row['Disadvantaged Player']) == float:
        return row['Unnamed: 4']
    else:
        return row['Disadvantaged Player']
    
df2['Disadvantaged Player'] = df2.apply(get_name, axis=1)

In [128]:
df2['Review Decision'].value_counts()

CNC    4043
CC     2019
INC     584
IC       62
Name: Review Decision, dtype: int64

In [129]:
df2['Period'].value_counts()

Q4          6689
Comment:    4741
mment:      2041
Q5           545
Q6            12
Q3             2
Name: Period, dtype: int64

In [130]:
df2.head()

,Period,Time Call Type,Committing Player,Disadvantaged Player,Unnamed: 4,Review Decision,Video,Date,Score
0,Q4,01:59.0 Foul: Defense 3 Second,Draymond Green,NaN,NaN,NaN,Video,0017-06-07,Warriors118_Cavaliers113
1,Comment:,[Observable in enhanced video] Green (GSW) is ...,NaN,NaN,NaN,NaN,NaN,0017-06-07,Warriors118_Cavaliers113
2,Q4,01:56.0 Foul: Shooting,Klay Thompson,Kyrie Irving,NaN,CNC,Video,0017-06-07,Warriors118_Cavaliers113
3,Comment:,Thompson (GSW) makes marginal contact with Irv...,NaN,NaN,NaN,NaN,NaN,0017-06-07,Warriors118_Cavaliers113
4,Q4,01:53.0 Foul: Personal,Kyrie Irving,Draymond Green,NaN,CNC,Video,0017-06-07,Warriors118_Cavaliers113


In [139]:
def format_df(df):
    try :
        df.drop('Unnamed: 4', axis=1, inplace=True)
        df.drop('Video', axis=1, inplace=True)
        df.columns = ['Period','data','Fouler','Foulee','Review','Date','Score']
    except : pass
    for idx, row in df.iterrows():
        if ('Comment:' in str(row['Period'])) | ('mment:' in str(row['Period'])):
            df.loc[idx-1, 'Comment'] = row['data']
#                 print(df.loc[idx-1, 'Comment'])
            df.drop(idx, axis=0, inplace=True)

    for idx, row in df.iterrows():
            df.loc[idx, 'Soup'] = ''
            for cell in row.values:
                df.loc[idx, 'Soup'] += str(cell) + ' '
            # Processing for column 'Time' and 'Foul' (foul type)
            match = re.search('\d\d:\d\d.\d', df.loc[idx, 'Soup'])
            if match:
                time = match.group()
                df.loc[idx, 'Time'] = time
            for foul in foul_types:
                if foul in df.loc[idx, 'Soup']:
                    df.loc[idx, 'Foul'] = foul

            review_done = (False if type(row['Review']) == float else True)
            if not review_done:  # if parser moved review decision to row below in table
                soup_temp = ''
                for cell in df.loc[idx+1,:].values:
                    soup_temp += str(cell) + ' '
                for review_decision in review_types:
                    if review_decision in soup_temp:
                        df_errors.loc[idx, 'Review Decision'] = review_decision  
                        review_done = True
            if not review_done:
                if 'Observable in enhanced video' in str(df.loc[idx, 'Comment']):
                    df_errors.loc[idx, 'Review Decision'] = 'PINC' # for Partial Incorrect No Call 
#     except Exception as err : print(err)


In [140]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8143 entries, 0 to 14099
Data columns (total 10 columns):
Period     8073 non-null object
data       8136 non-null object
Fouler     6820 non-null object
Foulee     5952 non-null object
Review     6706 non-null object
Date       8143 non-null object
Score      8143 non-null object
Soup       6410 non-null object
Time       6357 non-null object
Comment    5957 non-null object
dtypes: object(10)
memory usage: 1019.8+ KB


In [141]:
format_df(df2)
df2.info()
# Still many errors by tabula java function

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7318 entries, 0 to 14099
Data columns (total 11 columns):
Period     7248 non-null object
data       7311 non-null object
Fouler     6820 non-null object
Foulee     5952 non-null object
Review     6706 non-null object
Date       7318 non-null object
Score      7318 non-null object
Soup       7318 non-null object
Time       7253 non-null object
Comment    6782 non-null object
Foul       6272 non-null object
dtypes: object(11)
memory usage: 1006.1+ KB


In [144]:
df2['Foul'].value_counts()

Personal      2070
Shooting      1838
Offensive     1351
Loose Ball     584
Traveling      382
24 Second       47
Name: Foul, dtype: int64

In [146]:
df2['Review'].value_counts()

CNC    4043
CC     2017
INC     584
IC       62
Name: Review, dtype: int64

In [147]:
df2.to_csv('L2M_processed.csv', index=False, compression='gzip')